In [1]:
import numpy as np
import os
from skimage.io import imread
from skimage.filters import threshold_otsu
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras import optimizers
from keras.callbacks import EarlyStopping

In [2]:
letters = [
            '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D',
            'E', 'F', 'G', 'H','J', 'K', 'L', 'M', 'N','P', 'Q', 'R', 'S', 'T',
            'U', 'V', 'W', 'X', 'Y', 'Z'
        ]

letters_num = [i for i in range(0,len(letters))]

dictionary = dict(zip(letters, letters_num))

dict2 = dict(zip(letters_num, letters))

In [3]:
def read_training_data(training_directory):
    image_data =[]
    target_data=[]
    train_counter = 0
    img=[]
    amount = 3000
    for num in range(amount):
        for ltr in letters:
            img_path = os.path.join(training_directory, str(ltr), str(ltr) + '_' + str(num) + '.jpg')
            if os.path.exists(img_path)==True:
                try:
                    #print(img_path)
                    #img.append(np.array([1,2,3,4]))
                    img_details = imread(img_path, as_grey=True)
                    binary_image = img_details < threshold_otsu(img_details)
                    #print(binary_image.shape, type(binary_image))
                    flat_bin_image = binary_image.reshape(-1)
                    #flat_bin_image.tolist()
                    #print(type(flat_bin_image),flat_bin_image.shape )
                    #print(flat_bin_image.shape)
                    image_data.append(flat_bin_image)
                    #image_data.append(flat_bin_image)
                        #print(image_data)
                    target_data.append(ltr)
    
                except OSError as e:
                    os.remove(img_path)
                    print("Removed "+img_path)
            #train_counter+=1
    img=np.array(img)
    image_data = np.array(image_data)
    #image_data=image_data.flatten()
    target_data = np.array(target_data)
    #print(np.array(image_data).shape)
    print(image_data.shape,target_data.shape)       
    return (np.array(image_data), np.array(target_data))

In [4]:
current_dir = os.path.dirname(os.path.realpath("__file__"))

training_dataset_dir = os.path.join(current_dir, 'train/')
image_data, target_data = read_training_data(training_dataset_dir)

(68328,) (68328,)


In [5]:
fff=np.zeros((image_data.shape[0],400))
for i in range(image_data.shape[0]):
    fff[i,:] = image_data[0]

fff.shape

(68328, 400)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(fff, np.array(target_data), test_size=0.3, random_state=88)

In [6]:
X_train = X_train.reshape(X_train.shape[0],20, 20, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 20, 20,1).astype('float32')

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('tf')

Using TensorFlow backend.


In [10]:
temp = y_train
temp.shape
hold=[]
for tar in temp:
    #v=dict2[np.argmax(array)]
    #v=np.argmax(array)
    v=dictionary[tar]
    hold.append(v)
hold

temp2 = y_test
temp2.shape
hold2=[]
for array in temp2:
    v=dictionary[array]
    hold2.append(v)

47829

In [ ]:
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
#Xtrain = (X_train-np.mean(X_train, axis=0))/np.std(X_train, axis=0)
#Xtest = (X_test-np.mean(X_train, axis=0))/np.std(X_train, axis=0)
# one hot encode outputs
Y_train = np_utils.to_categorical(hold)
Y_test = np_utils.to_categorical(hold2)
num_classes = 34

In [20]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(20,20,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    #model.add(Dense(64), activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.001), metrics=['accuracy'])
    return model

In [ ]:
def model4():
    
    model = Sequential()
    model.add(Conv2D(32, (3,3),padding='valid', input_shape=(20,20,1), activation='relu'))
    model.add(Conv2D(64, (3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    model.add(Conv2D(128, (3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def larger_model():
	# create model
	model = Sequential()
	model.add(Conv2D(30, (5, 5), input_shape=(20,20,1), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Conv2D(15, (3, 3), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [27]:
def baseline_model():
    model = Sequential()
    model.add(Conv2D(16, (5, 5), input_shape=(20,20,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #model.add(Conv2D(32, (5, 5), activation='relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [30]:
def build_fit_model():
    # build the model
    model = baseline_model()
    #model = larger_model()
    # Fit the model
    cb=EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=0, mode='auto')
    history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=50, callbacks=[cb],batch_size=32, verbose=2)
    # Final evaluation of the model
    scores = model.evaluate(X_test, Y_test, verbose=0)
    print("CNN Error: %.2f%%" % (100-scores[1]*100))
    
    # serialize model to JSON
    model_json = model.to_json()
    with open("models/model_l.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("model_l.h5")
    print("Saved model to disk")
    
        # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [31]:
build_fit_model()

Train on 47829 samples, validate on 20499 samples
Epoch 1/50
16s - loss: 3.4973 - acc: 0.0298 - val_loss: 3.4925 - val_acc: 0.0312
Epoch 2/50
15s - loss: 3.4952 - acc: 0.0284 - val_loss: 3.4941 - val_acc: 0.0296
Epoch 3/50
16s - loss: 3.4948 - acc: 0.0328 - val_loss: 3.4927 - val_acc: 0.0301
Epoch 4/50
15s - loss: 3.4946 - acc: 0.0295 - val_loss: 3.4930 - val_acc: 0.0308
Epoch 5/50
15s - loss: 3.4947 - acc: 0.0295 - val_loss: 3.4938 - val_acc: 0.0308
Epoch 6/50
15s - loss: 3.4944 - acc: 0.0307 - val_loss: 3.4917 - val_acc: 0.0300
Epoch 7/50
14s - loss: 3.4943 - acc: 0.0297 - val_loss: 3.4918 - val_acc: 0.0304
Epoch 8/50
16s - loss: 3.4942 - acc: 0.0301 - val_loss: 3.4914 - val_acc: 0.0296
Epoch 9/50
15s - loss: 3.4940 - acc: 0.0305 - val_loss: 3.4918 - val_acc: 0.0308
Epoch 10/50
15s - loss: 3.4941 - acc: 0.0292 - val_loss: 3.4918 - val_acc: 0.0302
Epoch 11/50
16s - loss: 3.4938 - acc: 0.0304 - val_loss: 3.4910 - val_acc: 0.0300
Epoch 12/50
16s - loss: 3.4938 - acc: 0.0297 - val_loss: 

NameError: name 'plt' is not defined